In [ ]:
import numpy as np
import pandas as pd
import json
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

## Load PSPLib data

In [ ]:
with open(f"parsed_120/j12016_6.json", "r") as json_file:
    data = json.load(json_file)
    
    n_jobs = data["n_jobs"]
    N = data["n_jobs"]
    resource_pool = data["resource_pool"]
    best_known_solution = data['hrs']
    best_known_solution
    
    durations = {
        i['id_']: i['duration']
        for i in data['jobs']
    }

    predecessors = {
        i['id_']: i['predecessors']
        for i in data['jobs']
    }

    resources = {
        i['id_']: i['resources']
        for i in data['jobs']
    }

In [ ]:
best_known_solution

## SSGS

In [ ]:
class ResourcesManager:
    
    def __init__(self, horizon, start_pool):
        self.horizon = horizon
        self.start_pool = start_pool
        self.n_resources = len(start_pool)
        
        self.resource_timeline = np.ones((self.horizon, self.n_resources)) * np.array(self.start_pool)
                
    def get_is_avaliable(self, amount, from_, to_):
        answer = (self.resource_timeline[from_:to_, :] >= amount).all()
        return answer
    
    def allocate(self, amount, from_, to_):
        if not self.get_is_avaliable(amount, from_, to_):
            raise ValueError()
            
        self.resource_timeline[from_:to_, :] -= amount
        
    def allocate_earliest(self, amount, duration, starting_from):
        for i in range(starting_from, self.horizon+1):
            is_met = self.get_is_avaliable(amount, from_=i, to_=i+duration)
            if is_met:
                self.allocate(amount, from_=i, to_=i+duration)
                return i

In [ ]:
class PrecedenceManager:
    
    def __init__(self, predecessors):
        self.predecessors = predecessors
        self.is_completed = {
            i: False 
            for i in predecessors.keys()
        }
        
    def can_start(self, job_id):
        predecessors = self.predecessors[job_id]
        return all(
            self.is_completed[i]
            for i in predecessors
        )
    
    def what_can_start(self):
        return [
            i for i in self.predecessors.keys()
            if self.can_start(i) and not self.is_completed[i]
        ]
    
    def set_complete(self, job_id):
        self.is_completed[job_id] = True
        
    def are_all_completed(self):
        return all(self.is_completed.values())

In [ ]:
class ProjectPlan:
    
    def __init__(self, predecessors, resource_pool, durations, resources):
        self.durations = durations
        self.resources = resources
        self.rm = ResourcesManager(horizon=1000, start_pool=resource_pool)
        self.pm = PrecedenceManager(predecessors)
        self.gantt_chart = {
            i: None
            for i in predecessors.keys()
        }
        
    def add_to_schedule(self, job, at):
        self.pm.set_complete(job)
        self.gantt_chart[job] = (at, at + self.durations[job] - 1)   # at + self.durations[job] - 1
        
    def ssgs(self, priority_list):
        
        self.pm.set_complete(0)
        self.gantt_chart[0] = (0, 0)
        order_added = [0]
        
        while not self.pm.are_all_completed():
            candidates = self.pm.what_can_start()

            selected = max(candidates, key=lambda a: priority_list[a])
            order_added.append(selected)

            when_predecessors_complete = max(
                self.gantt_chart[i][1]
                for i in self.pm.predecessors[selected] or [0]
            )
    
            start_time = self.rm.allocate_earliest(
                self.resources[selected], 
                self.durations[selected], 
                when_predecessors_complete + 1
            )
            
            self.add_to_schedule(selected, start_time)
            
        return max(i[1] for i in self.gantt_chart.values())

In [ ]:
def get_time(priority_list):
    #print("Calculating...")
    return ProjectPlan(
        predecessors=predecessors, 
        resource_pool=resource_pool, 
        durations=durations, 
        resources=resources
    ).ssgs(priority_list)

## Baseline

In [ ]:
import functools
@functools.lru_cache(maxsize=1024)
def get_est(id_):
    if predecessors[id_] == []:
        return 0
    return max(
        get_est(p) + durations[p]
        for p in predecessors[id_]
    )

est = np.array([
    get_est(i)
    for i in range(n_jobs+2)
])
est

In [ ]:
baseline = min(get_time(-1*est), min(
    get_time(np.random.rand(122))
    for _ in range(250)
))
baseline

## Ants

In [ ]:
class AntSolution:
    """All about a solution: different representations and fitness"""
    
    def __init__(self, priority_pairs):
        self.priority_pairs = priority_pairs
        random_key = np.array([1000 * i[0] + i[1] for i in priority_pairs])  # maybe fix later
        self.random_key = random_key
        self.fitness = get_time(random_key)

In [ ]:
class pheromones:
    
    def __init__(self, n_bins, n_subbins):
        self.pheromone_matrix_list = [
            np.random.uniform(5, 5, (N+2, n_bins))
            for i in range(20)
        ]
        self.subpheromone_matrix_list = [
            np.random.uniform(5, 5, (N+2, n_bins, n_subbins))
            for i in range(20)
        ]
        
    def get_pheromones(self):
        return (
            sum(self.pheromone_matrix_list[-10:]),
            sum(self.subpheromone_matrix_list[-10:])
        )
    
    def add_pheromones(self, new_pheromones, new_subpheromones):
        self.pheromone_matrix_list.append(new_pheromones)
        self.subpheromone_matrix_list.append(new_subpheromones)

In [ ]:
class History:
    """Tracking generations and stats"""
    
    def __init__(self):
        self.generations = []
        self.current_generation = []
    
    def add_solution(self, solution):
        self.current_generation.append(solution)
        
    def end_generation(self):
        self.generations.append(self.current_generation)
        self.current_generation = []
        
    def get_stats(self):
        mins = [
            min(solution.fitness for solution in generation)
            for generation in self.generations
        ]
        mean = [
            sum(solution.fitness for solution in generation) / len(generation)
            for generation in self.generations
        ]
        return (mins, mean)
    
    def get_generation_stats(self):
        return (
            [i.fitness for i in self.current_generation],
            [i.priority_pairs for i in self.current_generation]
        )

In [ ]:
N = 120

class AntColony:
    
    def __init__(self, n_per_iteration=20, n_bins=5, n_subbins=5):
        
        self.n_bins = n_bins
        self.n_subbins = n_subbins
        self.n_per_iteration = n_per_iteration
        
        self.pheromones = pheromones(n_bins, n_subbins)
        self.history = History()
    
    @staticmethod
    def weighted_choice(array, weights):
        probabilities = weights / sum(weights)
        return np.random.choice(array, p=probabilities)
    
    @staticmethod
    def split_to_bins(array):
        bins = np.quantile(array, np.linspace(1, 0, 6))
        bins[0] = np.inf  # small fix for borders
        return np.digitize(array, bins=bins) - 2
    
    def get_one_solution(self):
        """Get one solution based on current pheromones"""
        
        pheromone_matrix, subpheromone_matrix = self.pheromones.get_pheromones()
        order = []
        for i in range(0, N+2):
            chosen_bin = self.weighted_choice(
                np.arange(self.n_bins), 
                pheromone_matrix[i]
            )
            chosen_subbin = self.weighted_choice(
                np.arange(self.n_subbins), 
                subpheromone_matrix[i][chosen_bin]
            )
            order.append([chosen_bin, chosen_subbin])
        return AntSolution(np.array(order))
    
    def get_one_generation(self):
        """Get pheromones from one generation"""
        
        for i in range(self.n_per_iteration):
            solution = self.get_one_solution()
            self.history.add_solution(solution)
                
        fitness_list, priority_pairs_list = self.history.get_generation_stats()
        total_pheromone, total_subpheromone = self.calculate_pheromones(fitness_list, priority_pairs_list)
        
        self.history.end_generation()
        return (total_pheromone, total_subpheromone)
    
    def calculate_pheromones(self, fitness_list, priority_pairs_list):
        """Get amount of pheromones to add"""
        
        total_pheromone = np.zeros([N+2, self.n_bins])
        total_subpheromone = np.zeros([N+2, self.n_bins, self.n_subbins])
        
        lengths_digitized = self.split_to_bins(fitness_list)
        for quantile_n, al in zip(lengths_digitized, priority_pairs_list):
            pheromone_amount = (quantile_n)**2 * np.sign(quantile_n)
            for node, value in enumerate(al):
                total_pheromone[node, value[0]] += pheromone_amount
                total_subpheromone[node, value[0], value[1]] += pheromone_amount
        return (
            np.clip(total_pheromone, 0, 10**6), 
            np.clip(total_subpheromone, 0, 10**6)
        )
    
    def update_pheromones(self, update_subpheromones=True):
        """Update pheromones"""
        new_pheromone, new_subpheromone = self.get_one_generation()
        self.pheromones.add_pheromones(new_pheromone, new_subpheromone)

In [ ]:
np.random.seed(0)

colony = AntColony(n_per_iteration=50, n_bins=5, n_subbins=5)

for j in range(10):

    for i in range(10):    
        colony.update_pheromones(update_subpheromones=False)
        print("-", end="")
        
    for i in range(10):    
        colony.update_pheromones(update_subpheromones=True)
        print("=", end="")
    
    print("#")

In [ ]:
mins, mean = colony.history.get_stats()

fig = px.scatter(y=[
    pd.Series(mins).cummin(), 
    mean,
    [baseline]*len(mins)
])

fig.update_layout(showlegend=False,
                  template="plotly_white",
                  xaxis_title='Поколение',                   
                  yaxis_title='Длина пути')
fig.update_traces(mode='lines', line_width=2)
fig.data[0].line.color = "black"
fig.data[1].line.color = "green"
fig.data[2].line.color = "lightgray"
# fig.update_layout(height=1080, width=1920, font_size=20)
fig.show()

In [ ]:
fig = px.imshow(
    colony.pheromones.pheromone_matrix_list[-1],
    color_continuous_scale=['black', 'darkred', 'red', 'orange', 'yellow', 'lime'])
fig.update_layout(template='plotly_dark', height=1200, width=400, coloraxis_showscale=False)
fig.show()